In [4]:
import socketserver
from http.server import SimpleHTTPRequestHandler as SimpleHandler
import json
import pandas as pd

## step 1: check the request
THis file also saved as datahandler.py file

In [1]:
PORT = 8080
FILENAME = '../../data/dSST.csv'


class WeatherDataHandler(SimpleHandler):
    def do_GET(self):
        if self.path.startswith('/data'):
            self.handle_weather_data_request()
        else:
            self.send_error(404, 'Not found')

    def handle_weather_data_request(self):
        if self.path == '/data/all':
            self.send_weather_data()
        else:
            path_parts = self.path.split('/')
            if len(path_parts) == 4 and path_parts[2].isdigit() and path_parts[3].isdigit():
                self.send_weather_data_range(int(path_parts[2]), int(path_parts[3]))
            elif len(path_parts) == 3 and path_parts[2].isdigit():
                self.send_weather_data_year(int(path_parts[2]))
            else:
                self.send_error(400, 'Bad Request')

    def send_weather_data(self):
        try:
            with open(FILENAME, 'r') as file:
                header = file.readline().strip().split(',')
                lines = file.readlines()

            json_data = []
            for line in lines:
                values = line.strip().split(',')
                json_dict = {key: value for key, value in zip(header, values)}
                json_data.append(json_dict)

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(json_data).encode())
        except FileNotFoundError:
            self.send_error(404, 'Not found')

    def send_weather_data_range(self, from_year, to_year):
        try:
            with open(FILENAME, 'r') as file:
                header = file.readline().strip().split(',')
                year_index = header.index('Year')
                lines = file.readlines()

            filtered_data = []
            for line in lines:
                values = line.strip().split(',')
                year = int(values[year_index])
                if from_year <= year <= to_year:
                    filtered_data.append(values)

            if not filtered_data:
                self.send_error(404, 'Not found')
                return

            json_data = []
            for values in filtered_data:
                json_dict = {key: value for key, value in zip(header, values)}
                json_data.append(json_dict)

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(json_data).encode())
        except FileNotFoundError:
            self.send_error(404, 'Not found')

    def send_weather_data_year(self, year):
        try:
            with open(FILENAME, 'r') as file:
                header = file.readline().strip().split(',')
                year_index = header.index('Year')

                for line in file:
                    values = line.strip().split(',')
                    if int(values[year_index]) == year:
                        json_dict = {key: value for key, value in zip(header, values)}
                        self.send_response(200)
                        self.send_header('Content-type', 'application/json')
                        self.end_headers()
                        self.wfile.write(json.dumps(json_dict).encode())
                        return

            self.send_error(404, 'Not found')
        except FileNotFoundError:
            self.send_error(404, 'Not found')


def main():
    with socketserver.TCPServer(("", PORT), WeatherDataHandler) as httpd:
        print("serving at port", PORT)
        httpd.serve_forever()


if __name__ == "__main__":
    main()


serving at port 8080


127.0.0.1 - - [05/Jul/2023 21:11:27] "GET /data/1950/1955 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2023 21:11:38] "GET /data/1950 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2023 21:11:45] "GET /data/all HTTP/1.1" 200 -


## step 2: create the data-provider


In [9]:
class DataProvider:
    '''
    The class is created to handle the data and generate the JSON response
    '''
    def __init__(self):
        FILENAME = '../../data/dSST.csv'
        self.data = pd.read_csv(FILENAME)

    def get_data(self, year_range=None):
        '''
        method takes an optional year_range parameter, which can be an integer representing a single year 
        or a list of two integers representing a range of years. It returns the filtered data as a JSON
        '''
        if year_range is None:
            return self.data.to_json(orient='records')

        if isinstance(year_range, int):
            filtered_data = self.data.loc[self.data['Year'] == year_range]
        elif isinstance(year_range, list) and len(year_range) == 2:
            filtered_data = self.data.loc[(self.data['Year'] >= year_range[0]) & (self.data['Year'] <= year_range[1])]
        else:
            raise ValueError("Invalid parameter")

        return filtered_data.to_json(orient='records')

In [11]:
d = DataProvider()
print(d.get_data()) # returns all the data in de csv as one json-stream

[{"Year":1881,"Jan":-0.18,"Feb":-0.13,"Mar":0.04,"Apr":0.06,"May":0.07,"Jun":-0.17,"Jul":0.02,"Aug":-0.02,"Sep":-0.14,"Oct":-0.21,"Nov":-0.17,"Dec":-0.06,"J-D":-0.07,"D-N":-0.08,"DJF":-0.16,"MAM":0.06,"JJA":-0.06,"SON":-0.17},{"Year":1882,"Jan":0.17,"Feb":0.15,"Mar":0.05,"Apr":-0.16,"May":-0.13,"Jun":-0.22,"Jul":-0.15,"Aug":-0.06,"Sep":-0.13,"Oct":-0.23,"Nov":-0.15,"Dec":-0.36,"J-D":-0.1,"D-N":-0.08,"DJF":0.09,"MAM":-0.08,"JJA":-0.15,"SON":-0.17},{"Year":1883,"Jan":-0.28,"Feb":-0.36,"Mar":-0.12,"Apr":-0.18,"May":-0.17,"Jun":-0.06,"Jul":-0.06,"Aug":-0.13,"Sep":-0.21,"Oct":-0.11,"Nov":-0.23,"Dec":-0.11,"J-D":-0.17,"D-N":-0.19,"DJF":-0.33,"MAM":-0.15,"JJA":-0.09,"SON":-0.19},{"Year":1884,"Jan":-0.12,"Feb":-0.07,"Mar":-0.36,"Apr":-0.39,"May":-0.33,"Jun":-0.34,"Jul":-0.32,"Aug":-0.27,"Sep":-0.26,"Oct":-0.24,"Nov":-0.33,"Dec":-0.3,"J-D":-0.28,"D-N":-0.26,"DJF":-0.1,"MAM":-0.36,"JJA":-0.31,"SON":-0.28},{"Year":1885,"Jan":-0.58,"Feb":-0.33,"Mar":-0.26,"Apr":-0.41,"May":-0.45,"Jun":-0.43,"Jul":

In [12]:
print(d.get_data(1991)) # returns one line of data; the one that corresponds to the year 1991


[{"Year":1991,"Jan":0.43,"Feb":0.5,"Mar":0.36,"Apr":0.51,"May":0.34,"Jun":0.53,"Jul":0.47,"Aug":0.39,"Sep":0.44,"Oct":0.29,"Nov":0.3,"Dec":0.32,"J-D":0.41,"D-N":0.41,"DJF":0.44,"MAM":0.4,"JJA":0.47,"SON":0.34}]


In [13]:
print(d.get_data([1991,2000])) # returns 10 lines of data, from the year 1991 to 2000

[{"Year":1991,"Jan":0.43,"Feb":0.5,"Mar":0.36,"Apr":0.51,"May":0.34,"Jun":0.53,"Jul":0.47,"Aug":0.39,"Sep":0.44,"Oct":0.29,"Nov":0.3,"Dec":0.32,"J-D":0.41,"D-N":0.41,"DJF":0.44,"MAM":0.4,"JJA":0.47,"SON":0.34},{"Year":1992,"Jan":0.48,"Feb":0.4,"Mar":0.48,"Apr":0.27,"May":0.31,"Jun":0.26,"Jul":0.09,"Aug":0.08,"Sep":-0.01,"Oct":0.06,"Nov":0.03,"Dec":0.21,"J-D":0.22,"D-N":0.23,"DJF":0.4,"MAM":0.36,"JJA":0.14,"SON":0.03},{"Year":1993,"Jan":0.35,"Feb":0.37,"Mar":0.36,"Apr":0.28,"May":0.29,"Jun":0.23,"Jul":0.25,"Aug":0.11,"Sep":0.12,"Oct":0.23,"Nov":0.04,"Dec":0.18,"J-D":0.23,"D-N":0.24,"DJF":0.31,"MAM":0.31,"JJA":0.2,"SON":0.13},{"Year":1994,"Jan":0.26,"Feb":0.03,"Mar":0.29,"Apr":0.42,"May":0.28,"Jun":0.44,"Jul":0.3,"Aug":0.22,"Sep":0.32,"Oct":0.42,"Nov":0.44,"Dec":0.38,"J-D":0.32,"D-N":0.3,"DJF":0.16,"MAM":0.33,"JJA":0.32,"SON":0.4},{"Year":1995,"Jan":0.53,"Feb":0.79,"Mar":0.47,"Apr":0.47,"May":0.27,"Jun":0.43,"Jul":0.45,"Aug":0.45,"Sep":0.34,"Oct":0.47,"Nov":0.44,"Dec":0.26,"J-D":0.45,"D-

## step 3: use the data-provider in the request handler
I also saved this as dataprovider_handler.py file.

In [15]:
PORT = 8080
FILENAME = '../../data/dSST.csv'

class WeatherDataHandler(SimpleHandler):
    def __init__(self, *args, **kwargs):
        self.data_provider = DataProvider()
        super().__init__(*args, **kwargs)

    def do_GET(self):
        if self.path.startswith('/data'):
            self.handle_weather_data_request()
        else:
            self.send_error(404, 'Not found')

    def handle_weather_data_request(self):
        try:
            if self.path == '/data/all':
                json_data = self.data_provider.get_data()
            else:
                path_parts = self.path.split('/')
                if len(path_parts) == 4 and path_parts[2].isdigit() and path_parts[3].isdigit():
                    from_year, to_year = int(path_parts[2]), int(path_parts[3])
                    json_data = self.data_provider.get_data([from_year, to_year])
                elif len(path_parts) == 3 and path_parts[2].isdigit():
                    year = int(path_parts[2])
                    json_data = self.data_provider.get_data(year)
                else:
                    self.send_error(400, 'Bad Request')
                    return

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json_data.encode())
        except ValueError as e:
            self.send_error(400, str(e))
        except FileNotFoundError:
            self.send_error(404, 'Not found')


def main():
    with socketserver.TCPServer(("", PORT), WeatherDataHandler) as httpd:
        print("serving at port", PORT)
        httpd.serve_forever()


if __name__ == "__main__":
    main()

serving at port 8080


127.0.0.1 - - [05/Jul/2023 21:43:25] "GET /data/all HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2023 21:43:40] "GET /data/1955 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jul/2023 21:43:56] "GET /data/1955/2000 HTTP/1.1" 200 -


## Exercise 2: create an async client

This part is done on the network_client.py file